In [ ]:
import os
import s3fs
os.environ["AWS_ACCESS_KEY_ID"] = 'BFMNK1LUPXQ061C6PZ55'
os.environ["AWS_SECRET_ACCESS_KEY"] = '1lUWhLj95YhcD8P5t6GEMXxwlf0jyzmqrGPMURQE'
os.environ["AWS_SESSION_TOKEN"] = 'eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJhY2Nlc3NLZXkiOiJCRk1OSzFMVVBYUTA2MUM2UFo1NSIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sImF1ZCI6WyJtaW5pby1kYXRhbm9kZSIsIm9ueXhpYSIsImFjY291bnQiXSwiYXV0aF90aW1lIjoxNzU5MDQ2Njc3LCJhenAiOiJvbnl4aWEiLCJlbWFpbCI6Imx1Y2FzLmN1bXVuZWxAZW5zYWUuZnIiLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiZXhwIjoxNzU5NjUxNTE1LCJmYW1pbHlfbmFtZSI6IkN1bXVuZWwiLCJnaXZlbl9uYW1lIjoiTHVjYXMiLCJncm91cHMiOlsiVVNFUl9PTllYSUEiLCJzdGF0YXBwLXNlZ21lZGljIl0sImlhdCI6MTc1OTA0NjcxNSwiaXNzIjoiaHR0cHM6Ly9hdXRoLmxhYi5zc3BjbG91ZC5mci9hdXRoL3JlYWxtcy9zc3BjbG91ZCIsImp0aSI6Im9ucnRydDplNzkxZWViNS1kYWQyLWIxZjYtMzJjZi05YTM2ODVjNTNlNGIiLCJuYW1lIjoiTHVjYXMgQ3VtdW5lbCIsInBvbGljeSI6InN0c29ubHkiLCJwcmVmZXJyZWRfdXNlcm5hbWUiOiJsYWIiLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsib2ZmbGluZV9hY2Nlc3MiLCJ1bWFfYXV0aG9yaXphdGlvbiIsImRlZmF1bHQtcm9sZXMtc3NwY2xvdWQiXX0sInJlc291cmNlX2FjY2VzcyI6eyJhY2NvdW50Ijp7InJvbGVzIjpbIm1hbmFnZS1hY2NvdW50IiwibWFuYWdlLWFjY291bnQtbGlua3MiLCJ2aWV3LXByb2ZpbGUiXX19LCJyb2xlcyI6WyJvZmZsaW5lX2FjY2VzcyIsInVtYV9hdXRob3JpemF0aW9uIiwiZGVmYXVsdC1yb2xlcy1zc3BjbG91ZCJdLCJzY29wZSI6Im9wZW5pZCBwcm9maWxlIGdyb3VwcyBlbWFpbCIsInNpZCI6ImFmY2VhMjNmLTM3MGUtNDhjNC1hNGQ5LWIzZDJjMDJkYmNkOCIsInN1YiI6ImUyZDc4NjRjLTcwMzItNDI0ZC04OTA2LWU0ZjhiNDFjYzAwMyIsInR5cCI6IkJlYXJlciJ9.JF4l6bNSZ2jJniDyrrDJRipM1GFJh5a05vfMLSwOcL5vXJbg6BjsZ639IP1NSk2phHnYPPnpD8xnCk6Pub1mEw'
os.environ["AWS_DEFAULT_REGION"] = 'us-east-1'
fs = s3fs.S3FileSystem(
    client_kwargs={'endpoint_url': 'https://'+'minio.lab.sspcloud.fr'},
    key = os.environ["AWS_ACCESS_KEY_ID"], 
    secret = os.environ["AWS_SECRET_ACCESS_KEY"], 
    token = os.environ["AWS_SESSION_TOKEN"])

path = "lab/dblp (1).xml.gz"


In [ ]:
%pip install lxml
import gzip
import xml.etree.ElementTree as ET
from lxml import etree
import pandas as pd


articles = []

with fs.open(path, 'rb') as f:
    with gzip.GzipFile(fileobj=f) as gz:
        #parser = etree.XMLParser(recover=True, resolve_entities=True)
        context = etree.iterparse(gz, events=("end",), tag="article", recover=True)
        
        for event, elem in context:
            title = elem.findtext("title")
            journal = elem.findtext("journal")
            pub_date = elem.findtext("pub_date")

            authors = [a.text for a in elem.findall(".//author")]
            authors_str = "; ".join([a for a in authors if a])

            articles.append({
                "title": title,
                "journal": journal,
                "pub_date": pub_date,
                "authors": authors_str
            })

            elem.clear()

df = pd.DataFrame(articles)

In [ ]:
chunks = []
n_chunks = 50
chunk_size = len(df) // n_chunks

for i in range(n_chunks):
    start = i * chunk_size
    end = (i + 1) * chunk_size if i < n_chunks - 1 else len(df)  # last chunk includes remainder
    
    chunk = df.iloc[start:end].copy()
    
    # Split authors column into multiple columns
    #authors_df = chunk['authors'].str.split(';', expand=True)
    #authors_df.columns = [f'author{i+1}' for i in range(authors_df.shape[1])]
    
    # Combine with original chunk
    #chunk = pd.concat([chunk.drop(columns=['authors']), authors_df], axis=1)
    
    chunk.to_parquet(f"t/articles_chunk_{i}.parquet", index=False)

In [ ]:

import pandas as pd
from collections import defaultdict
import glob

# Dictionary to collect coauthors per ego
ego_dict = defaultdict(set)

# List of all your chunk files
chunk_files = glob.glob("t/articles_chunk_*.parquet")

for file in chunk_files:
    df_chunk = pd.read_parquet(file)
    
    for authors_str in df_chunk['authors']:
        authors = authors_str.split(';')
        for ego in authors:
            coauthors = set(authors) - {ego}
            ego_dict[ego].update(coauthors)
    
    # Free memory after each chunk
    del df_chunk


In [ ]:
ego_df = pd.DataFrame([
    {'ego': ego, 'coauthors': ",".join(sorted(list(coauthors)))}
    for ego, coauthors in ego_dict.items()
])


In [ ]:
ego_df.to_parquet("ego_coauthors.parquet", index=False)
s3_path = "s3://lab/ego_coauthors.parquet"  # bucket + key

# Save directly to MinIO
ego_df.to_parquet(s3_path, engine="pyarrow", index=False, filesystem=fs)